In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import time

In [43]:
# Helper function to clean and preprocess text
def clean_text(text):
    # Removing HTML tags and special characters
    text = re.sub('<[^<]+?>', '', text)
    text = re.sub('[\r\n\t]', ' ', text)
    text = re.sub(' +', ' ', text)  # Removing extra spaces
    return text.strip()

# Function to crawl main text from a URL
def crawl_main_text(url):
    try:
        req = requests.get(url)
        req.encoding = None
        soup = BeautifulSoup(req.text, 'html.parser')
        
        # Find article content
        article_body = soup.find_all(['p', 'div'], class_=re.compile(r'(article|content|body|txt|news)', re.I))
        article_text = ' '.join([clean_text(p.text) for p in article_body if p and p.text])
        
        return article_text

    except Exception as e:
        print(f"Error while crawling {url}: {e}")
        return ""

# Function to crawl news articles based on a keyword
def crawl_news_articles(keyword, max_pages=10):
    articles = []
    for page in range(1, max_pages + 1):
        search_url = f"https://search.naver.com/search.naver?where=news&query={keyword}&start={(page-1)*10+1}"
        response = requests.get(search_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extracting news articles
        news_list = soup.select('div.group_news > ul.list_news > li')
        for news in news_list:
            title_tag = news.select_one('a.news_tit')
            title = title_tag.text if title_tag else None
            link = title_tag['href'] if title_tag else None
            date_tag = news.select_one('.info_group > span.info')
            date = date_tag.text if date_tag else None
            
            if link:
                article_text = crawl_main_text(link)
                if article_text:
                    articles.append({
                        'Keyword': keyword,
                        'Title': title,
                        'Link': link,
                        'Date': date,
                        'Content': article_text
                    })
        
        time.sleep(1)  # Too many requests 방지
    
    return articles

# 검색 키워드 리스트
keywords = ['니켈 원자재', '니켈 배터리', '니켈 전기차 배터리']

# 모든 키워드에 대해 기사 검색 및 수집
all_articles = []
for keyword in keywords:
    articles = crawl_news_articles(keyword, max_pages=3)  # 각 키워드당 3페이지 크롤링
    all_articles.extend(articles)

# DataFrame으로 변환
df = pd.DataFrame(all_articles)

# 크롤링 결과 확인
print(f"총 {len(df)}개의 기사가 크롤링되었습니다.")
print(df.head())

총 72개의 기사가 크롤링되었습니다.
  Keyword                                      Title  \
0  니켈 원자재                           원자재 가격 현황 6월 25일   
1  니켈 원자재  달러화 강세 지속…강달러 독주 우려 [최보화의 글로벌 ETF·원자재 시황]   
2  니켈 원자재                     에코프로비엠 '바닥론' 솔솔 나오는 이유   
3  니켈 원자재                      프랑스, 희토류·리튬·니켈 확보전 속도   
4  니켈 원자재             캐즘 엎친데 원자재 덮친격…K-배터리, 불황 돌파구는?   

                                                Link    Date  \
0  https://www.bntnews.co.kr/article/view/bnt2024...  11시간 전   
1  https://www.wowtv.co.kr/NewsCenter/News/Read?a...    1일 전   
2  http://www.smedaily.co.kr/news/articleView.htm...    1일 전   
3  https://www.theguru.co.kr/news/article.html?no...    4일 전   
4  http://www.m-i.kr/news/articleView.html?idxno=...    5일 전   

                                             Content  
0  Economy 원자재 가격 현황 6월 25일 김진아 기자 2024-06-25 14:...  
1  HOT 보잉, 스피릿에어로 주당 35달러에 인수 제안 與 국회부의장 주호영·박덕흠 ...  
2  상단영역 편집 : 2024-06-25 22:18 (화) 이슈모아보기 종합 산업 전기...  
3  전체기사 경제 산업 금융·증권 건설 생활경제 IT·스타트업 글

In [6]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

start_date = "2024.02.01"
end_date = "2024.04.30"
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
#https://search.naver.com/search.naver?where=news&query={keyword}&pd=3&ds={start_date}&de={end_date}&start={page}
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start="  + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:리튬 전기차 배터리

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):50

크롤링할 종료 페이지:  50 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=1', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=11', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=21', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=31', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=41', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=51', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=61', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&

100%|███████████████████████████████████████████| 60/60 [00:14<00:00,  4.01it/s]

검색된 기사 갯수: 총  500 개

[뉴스 제목]
["삼성SDI, 전기차 둔화에 '삐끗'…홀로 흑자는 빛났다", '삼성SDI, 전기차 둔화에도 1분기 \'선방\'…"투자 더 늘리겠다"(종합)', '투자 더 늘린다는 삼성SDI "전기차 장기 전망 밝아"', "K-배터리, 씁쓸한 1분기 실적…'전기차 캐즘' 영향 뚜렷", '사업부별 엇갈린 실적… SK이노, 정유 웃고 배터리 울다', "'전기차 캐즘'에 삼성SDI만 흑자…“신차 출시에 반전”", "석화·배터리 동반 부진… LG화학, 영업익 '4분의1 토막'", '‘꿈의 배터리’ 전고체 한중일 상용화 전쟁', "'전기차 캐즘'에 삼성SDI만 흑자…“신차 출시에 반전”", '[단독]포스코 장인화號, 배터리 소재 후퇴 없다…美서 리튬 사업 추진', '폴스타 CEO "고성능 전기차, 달리는 즐거움에 더 집중"', '엑셀 밟는 비야디 전기차…"연내 승용차 브랜드 론칭"', '부천도시공사, 전기차 화재 대비하는 수벽관창 설치한다', '중국산 전기차·태양광·리튬이온 쓰나미…‘차이나 쇼크 2.0’ 시작', "'3천만원대' 기아 EV3 6월 출시…전기차 '캐즘' 돌파 선봉장", 'K배터리 3사 실적 악화 지속…믿을 건 美 IRA뿐', '"배터리 가격 경쟁력 갖추자"…이석희 SK온 사장 \'승부수\'', '[기자수첩]배터리 투자 줄이는 韓, 후퇴는 안 된다', 'K-배터리의 ‘아픈 손가락’…SK온, 이번에도 적자 행진', 'SES AI, 현대차·기아와 리튬메탈전지 B샘플 공동개발', '전기차 배터리 소재 리튬값 다시 오른다', 'LG엔솔, 1분기 美 IRA 빼면 316억 ‘적자’…전기차 둔화 영향(상보)', 'LG엔솔 "투자 규모 조정하겠다"…\'전기차 캐즘\' 직격탄', "'무임승차'에 뿔난 LG엔솔 '배터리 특허 전쟁' 선언…中 겨냥했나", "포스코홀딩스, 1분기 영업익 17.3% 감소…배터리 투자 '브레이크'", '車업계, 전기차 신차 잇단 출시…캐즘 정면돌파 의지', "BTS 정국車 'G바겐' 전기차 나왔다…벤츠, 중국서 

In [3]:
#df1 = pd.read_csv('니켈 배터리_20240626_02시00분37초.csv')#리튬 배터리
df1= news_df.copy()

In [5]:
#df2 = pd.read_csv('니켈 원자재_20240626_02시01분47초.csv') #리튬 원자재
df2= news_df.copy()

In [7]:
#df3 = pd.read_csv('니켈 전기차 배터리_20240626_02시02분31초.csv') #리튬 전기차 배터리
df3= news_df.copy()

In [8]:
import re

def clean_text(text):
    # HTML 태그 제거
    text = re.sub('<[^<]+?>', '', text)
    # 줄바꿈, 탭 문자 제거
    text = re.sub('[\r\n\t]', ' ', text)
    # 특수문자 제거 (예: ', ", -, ...)
    text = re.sub(r'[^a-zA-Z0-9가-힣\s]', '', text)
    # 중복 공백 제거
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


In [9]:
df1['Content'] = df1['content'].apply(clean_text)
df1 = df1[['date','title','Content']]
df2['Content'] = df2['content'].apply(clean_text)
df2 = df2[['date','title','Content']]
df3['Content'] = df3['content'].apply(clean_text)
df3 = df3[['date','title','Content']]

In [10]:
df1['Keyword'] = '리튬 배터리'
df2['Keyword'] = '리튬 원자재'
df3['Keyword'] = '리튬 전기차 배터리'

In [16]:
# 검색 키워드 리스트
keywords = ['리튬 배터리', '리튬 원자재', '리튬 전기차 배터리']

In [11]:
# 데이터프레임을 리스트로 묶어서 concat 함수에 전달하여 합침
df = pd.concat([df1, df2, df3], ignore_index=True)
df

,date,title,Content,Keyword
0,2024-04-29 08:33:01,"中 CATL ""2027년 전고체배터리 소량 생산""",아직 원가 등 문제로 상용화 불가 입장 밝혀중국 배터리 기업 CATL이 전기차용 전...,리튬 배터리
1,2024-04-29 11:07:45,[단독] 포스코·LX인터 등 韓기업 의기투합…칠레 리튬 입찰에 '팀코리아',광해광업공단 등 민관합동 사업조사단한칠레 자원협력위원회 12년만에 재개광업부 장관 ...,리튬 배터리
2,2024-04-30 18:16:03,"삼성SDI ""올 공격 투자""…배터리 3사 유일",삼성SDI가 배터리셀 제조 3사 중 유일하게 올해 1분기 흑자를 기록했다 이익 규모...,리튬 배터리
3,2024-04-29 11:00:00,"[단독]포스코 장인화號, 배터리 소재 후퇴 없다…美서 리튬 사업 추진",보죠배터리캘리포니아 솔턴 호 인근서 지열발전 결합 리튬 추출 사업 포스코홀딩스 리튬...,리튬 배터리
4,2024-04-29 15:00:01,"'中 깜짝 방문' 머스크, 배터리 업체 CATL 회장과도 조우",테슬라 주요 배터리 공급업체 CATL저가 전기차 탑재 배터리 논의 추정머스크 인도 ...,리튬 배터리
...,...,...,...,...
146,2024-04-19 09:49:01,“육각형 전기차 뜬다”...전기 SUV 쿠페 ‘폴스타 4’ 6월 출시,폴스타 2 이어 국내 두 번째 선보이는 모델공식 출시 후 4개월 뒤인 10월부터 인...,리튬 전기차 배터리
147,2024-04-25 14:53:01,배터리 권위자 셜리 멍 “성능 떨어져도 안전성 문제 해결해야...무음극 나트륨 전고...,24일 제주서 한국화학공학회 학술대회셜리 멍 미국 시카고대 교수 기조강연 무음극 나...,리튬 전기차 배터리
148,2024-04-25 12:26:26,"LG엔솔 ""2분기도 수요회복 시간 걸려…하반기 실적개선 가능성""",하반기 신규모델 출시 확대GM JV 2기 램프업 등 긍정적 요인투자 선택과 집중우선...,리튬 전기차 배터리
149,2024-04-26 17:51:08,"中 배터리굴기 과시…""세계 첫 반고체 양산車 내놔""",오토차이나 2024 홈그라운드서 신기술 대거 선봬CATL 고성능 인산철 배터리1회 ...,리튬 전기차 배터리


In [12]:
df.to_csv('df_리튬2.csv', index = False)

In [20]:
# 한국어 불용어 리스트 (필요에 따라 추가/제거 가능)
korean_stopwords = ['것으로', '올해','있는','대한','이후','지난해',
    '이', '그', '저', '것', '있', '하', '것들', '들', '에서', '의', '를', '으로', '에', '그리고', '가', '를', '에게',
    '뿐만', '뿐', '이다', '있다', '없다', '에서', '과', '도', '을', '와', '이다', '으로', '이다', '것이다', '더', '등',
    '만', '이다', '또', '한', '가장', '것', '때문에', '로', '에', '이', '들', '잘', '한', '수', '하다'
]


In [21]:
# TF-IDF 분석 및 중요 단어 추출
def tfidf_analysis(df, keywords):
    results = {}
    tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words=korean_stopwords)
    
    for keyword in keywords:
        keyword_df = df[df['Keyword'] == keyword].reset_index(drop=True)
        if not keyword_df.empty:
            tfidf_matrix = tfidf_vectorizer.fit_transform(keyword_df['Content'])
            feature_names = tfidf_vectorizer.get_feature_names_out()
            tfidf_df = pd.DataFrame(tfidf_matrix.T.toarray(), index=feature_names)
            tfidf_df.columns = keyword_df['title']
            
            # 중요 단어 추출
            keyword_tfidf = tfidf_df.sum(axis=1)
            top_terms = keyword_tfidf.sort_values(ascending=False).head(10)
            
            results[keyword] = top_terms
    
    return results

In [22]:
# TF-IDF 분석
tfidf_results = tfidf_analysis(df, keywords)

In [23]:
# 결과 출력
for keyword, top_terms in tfidf_results.items():
    print(f"\n키워드: {keyword}\n")
    print(top_terms)


키워드: 리튬 배터리

배터리     5.750440
전기차     3.143771
중국      2.831411
리튬      2.732566
1분기     2.501140
미국      2.226009
전고체     2.056766
배터리를    2.041040
세계      1.672574
투자      1.619847
dtype: float64

키워드: 리튬 원자재

리튬     3.312734
배터리    2.492911
가격이    2.225144
가격     2.161614
원자재    1.732021
미국     1.691345
전기차    1.687729
1분기    1.663883
실적     1.470844
가격은    1.460495
dtype: float64

키워드: 리튬 전기차 배터리

배터리     6.045710
전기차     4.425403
1분기     3.424224
중국      2.973797
미국      2.757627
수요      2.440944
리튬      2.273356
대비      2.243116
시장      2.193721
배터리를    2.022230
dtype: float64


In [24]:
# 모든 DataFrame 및 분석 결과 저장
df.to_csv('all_news_articles.csv', index=False)
for keyword in keywords:
    keyword_df = df[df['Keyword'] == keyword]
    keyword_df.to_csv(f'news_articles_{keyword}.csv', index=False)
    if keyword in tfidf_results:
        tfidf_results[keyword].to_csv(f'top_terms_{keyword}.csv')

# 리튬 키워드 : 리튬 전기차 배터리

In [31]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

start_date = "2024.02.01"
end_date = "2024.04.30"
# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
#https://search.naver.com/search.naver?where=news&query={keyword}&pd=3&ds={start_date}&de={end_date}&start={page}
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&query=" + search + f"&pd=3&ds={start_date}&de={end_date}&start="  + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)

검색할 키워드를 입력해주세요:리튬 전기차 배터리

크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):400

크롤링할 종료 페이지:  400 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=1', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=11', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=21', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=31', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=41', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=51', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=3&ds=2024.02.01&de=2024.04.30&start=61', 'https://search.naver.com/search.naver?where=news&query=리튬 전기차 배터리&pd=

100%|███████████████████████████████████████████| 90/90 [00:20<00:00,  4.38it/s]

검색된 기사 갯수: 총  4000 개

[뉴스 제목]
["삼성SDI, 전기차 둔화에 '삐끗'…홀로 흑자는 빛났다", '삼성SDI, 전기차 둔화에도 1분기 \'선방\'…"투자 더 늘리겠다"(종합)', '투자 더 늘린다는 삼성SDI "전기차 장기 전망 밝아"', "K-배터리, 씁쓸한 1분기 실적…'전기차 캐즘' 영향 뚜렷", '사업부별 엇갈린 실적… SK이노, 정유 웃고 배터리 울다', "'전기차 캐즘'에 삼성SDI만 흑자…“신차 출시에 반전”", "석화·배터리 동반 부진… LG화학, 영업익 '4분의1 토막'", '‘꿈의 배터리’ 전고체 한중일 상용화 전쟁', "'전기차 캐즘'에 삼성SDI만 흑자…“신차 출시에 반전”", '[단독]포스코 장인화號, 배터리 소재 후퇴 없다…美서 리튬 사업 추진', '폴스타 CEO "고성능 전기차, 달리는 즐거움에 더 집중"', '엑셀 밟는 비야디 전기차…"연내 승용차 브랜드 론칭"', '부천도시공사, 전기차 화재 대비하는 수벽관창 설치한다', '중국산 전기차·태양광·리튬이온 쓰나미…‘차이나 쇼크 2.0’ 시작', "'3천만원대' 기아 EV3 6월 출시…전기차 '캐즘' 돌파 선봉장", 'K배터리 3사 실적 악화 지속…믿을 건 美 IRA뿐', '"배터리 가격 경쟁력 갖추자"…이석희 SK온 사장 \'승부수\'', '[기자수첩]배터리 투자 줄이는 韓, 후퇴는 안 된다', 'K-배터리의 ‘아픈 손가락’…SK온, 이번에도 적자 행진', 'SES AI, 현대차·기아와 리튬메탈전지 B샘플 공동개발', '전기차 배터리 소재 리튬값 다시 오른다', 'LG엔솔, 1분기 美 IRA 빼면 316억 ‘적자’…전기차 둔화 영향(상보)', 'LG엔솔 "투자 규모 조정하겠다"…\'전기차 캐즘\' 직격탄', "'무임승차'에 뿔난 LG엔솔 '배터리 특허 전쟁' 선언…中 겨냥했나", "포스코홀딩스, 1분기 영업익 17.3% 감소…배터리 투자 '브레이크'", '車업계, 전기차 신차 잇단 출시…캐즘 정면돌파 의지', "BTS 정국車 'G바겐' 전기차 나왔다…벤츠, 중국서

In [26]:
df1 = news_df.copy() #100행

In [28]:
df2 = news_df.copy() #200행

In [30]:
df3 = news_df.copy() #300행

In [32]:
df4 = news_df.copy() #400행

In [33]:
# 각 DataFrame의 행 개수를 구하기
row_count_df1 = len(df1)
row_count_df2 = len(df2)
row_count_df3 = len(df3)
row_count_df4 = len(df4)

# 가장 많은 행 수를 가진 DataFrame을 출력
max_rows_df = max([(row_count_df1, 'df1'), (row_count_df2, 'df2'), (row_count_df3, 'df3'), (row_count_df4, 'df4')])[1]
print(f"가장 많은 행 수를 가진 DataFrame은 {max_rows_df}입니다.")

가장 많은 행 수를 가진 DataFrame은 df3입니다.


In [34]:
print(df1.shape, df2.shape, df3.shape, df4.shape)

(85, 4) (80, 4) (104, 4) (87, 4)


In [35]:
# 1. pDate 열을 datetime 형식으로 변환
df3['date'] = pd.to_datetime(df3['date'])
# 2. pDate 열을 기준으로 데이터프레임을 날짜순으로 정렬
df3 = df3.sort_values(by='date')

df3.to_csv('리튬_뉴스크롤링_최종.csv', index = False)

In [36]:
df3

,date,title,link,content
80,2024-02-01 13:15:04,"중국, 전기차 수요 감소로 리튬기업 지난해 순이익 70%↓",https://n.news.naver.com/mnews/article/016/000...,[\n올해와 내년에도 리튬 생산 과잉이 예상…추가 하락할 듯리튬 배터리 수요 증가율...
98,2024-02-06 13:57:22,美·中 견제 강화한 韓 전기차 보조금…배터리·AS 강점 국산차 비교우위,https://n.news.naver.com/mnews/article/421/000...,"[\nLFP배터리 보조금 삭감, 정보수집장치 未탑재 테슬라도 영향'주행거리·충전속도..."
90,2024-02-06 15:19:01,"""싼 차를 비싸게 사라고?"" 배터리 성능 따른 전기차 보조금 지급의 모순",https://n.news.naver.com/mnews/article/018/000...,"[\n환경부, 6일 '2024 전기차 보조금 개편 방안' 공개배터리 밀도 등 배터리..."
75,2024-02-12 17:29:01,"황화 리튬 양산…""차세대 배터리 소재 주도""",https://n.news.naver.com/mnews/article/015/000...,[\n레이크머티리얼즈의 야심LED 전구체·TMA 글로벌 1위고부가가치 제품으로 이익...
73,2024-02-14 08:39:01,"“전기차 27만대 생산 분량”...LG엔솔, 호주서 리튬 공급망 강화",https://n.news.naver.com/mnews/article/009/000...,[\n\n\n\n\n [사진출처 = LG에너지솔루션]LG에너지솔루션이 고용량 전기차...
...,...,...,...,...
0,2024-04-30 13:51:01,"삼성SDI, 전기차 둔화에 '삐끗'…홀로 흑자는 빛났다",https://n.news.naver.com/mnews/article/648/000...,[\n[워치전망대]1Q 영업이익 전년比 28.8% ↓흑자 기조는 이어가…'P5·P6...
11,2024-04-30 15:48:40,"부천도시공사, 전기차 화재 대비하는 수벽관창 설치한다",https://n.news.naver.com/mnews/article/008/000...,[\n\n\n\n\n원명희 사장이 29일 관내 주차장에 직접 수벽관창 시험을 했다....
5,2024-04-30 16:43:01,'전기차 캐즘'에 삼성SDI만 흑자…“신차 출시에 반전”,https://n.news.naver.com/mnews/article/011/000...,"[\n■1분기 부진 K배터리, 회복 시점은영업익 29% 감소하며 '상대적 선방'LG..."
14,2024-04-30 16:46:03,K배터리 3사 실적 악화 지속…믿을 건 美 IRA뿐,https://n.news.naver.com/mnews/article/092/000...,"[\n1Q 성적표 부진 속 전기차 '캐즘' 속 활로 모색LG에너지솔루션, 삼성SDI..."
